In [1]:
# partial correlation of SOS with
# environmental drivers: 1. temperature, 2. precipitation, 3. solar radiation, 4. soil moisture and 5. atmospheric CO2
# we consider two preseason windows to aggregate environmental factors - Spring (MAM) and Winter(DJF)
# same years first semiannual CO2 image is used for both windows

import os
import numpy as np  
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import rioxarray as rxr
import pingouin as pg   


In [ ]:
#load significant sos raster and filter for valid SUs
sig_trend_rxr = rxr.open_rasterio(r"../Data/Trend_Rasters/mod_sos_mk_significant.tif")
valid_lsp_rxr = rxr.open_rasterio(r"../Data/Processed/Valid_lsp_raster/Valid_lsp_change_sos.tif")

#mask sig_trend rxr to pixel locations where lsp change is valid
valid_trend_rxr = sig_trend_rxr.where(valid_lsp_rxr == 1)


[[[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]]
(1, 1967, 3930)


In [ ]:
#make an xr stack of all monthly environmental variables raster + semi-annual CO2 raster + 